In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime

#load files
train = pd.read_csv('../input/train.csv', parse_dates=['timestamp'])
test = pd.read_csv('../input/test.csv', parse_dates=['timestamp'])
macro = pd.read_csv('../input/macro.csv', parse_dates=['timestamp'])
id_test = test.id

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=np.log1p(train.life_sq.values), y=np.log1p(train.price_doc.values), size=10,kind="hex")
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Log of living area in square metre', fontsize=12)
plt.show()

In [ ]:
Q1 = np.percentile(np.log1p(train.price_doc.values), 25)
Q2 = np.percentile(np.log1p(train.price_doc.values), 50)
Q3 = np.percentile(np.log1p(train.price_doc.values), 75)
IQR=Q3 - Q1
infbdd=Q1 - 1.5 * IQR 
supbdd=Q3 + 1.5 * IQR 
ibdd=int(np.exp(13))
sbdd=int(2.5 * np.exp(supbdd))
sindex=train[train.price_doc > sbdd].index
iindex=train[train.price_doc < ibdd].index
train.ix[iindex,"price_doc"] = ibdd
train.ix[sindex,"price_doc"] = sbdd

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=np.log1p(train.life_sq.values), y=np.log1p(train.price_doc.values), size=10,kind="hex")
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Log of living area in square metre', fontsize=12)
plt.show()

In [ ]:
equal_index = [601,1896,2791]
test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]

In [ ]:
bad_index = train[train.life_sq < 2].index
train.ix[bad_index, "life_sq"] = np.NaN

bad_index = test[test.life_sq < 2].index
test.ix[bad_index, "life_sq"] = np.NaN

bad_index = train[train.full_sq < 2].index
train.ix[bad_index, "full_sq"] = np.NaN

bad_index = test[test.full_sq < 2].index
test.ix[bad_index, "full_sq"] = np.NaN

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=np.log1p(train.life_sq.values), y=np.log1p(train.price_doc.values), size=10,kind="hex")
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Log of living area in square metre', fontsize=12)
plt.show()

In [ ]:
bad_index = test[test.life_sq > test.full_sq]
bad_index

In [ ]:
exchange_index=[64,119,171]
life_bad_index=[2027, 2031, 5187]
full_bad_index=[2804]
test.ix[life_bad_index, "life_sq"] = np.NaN
test.ix[full_bad_index, "full_sq"] = np.NaN
for cat in exchange_index:
    dog=test.ix[cat, "life_sq"]
    test.ix[cat, "life_sq"] = test.ix[cat, "full_sq"]
    test.ix[cat, "full_sq"]=dog
    

In [ ]:
test.ix[exchange_index]

In [ ]:
print(np.nanpercentile(np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values), 99.9))
print(np.nanpercentile(np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values), 0.1))
print(np.nanpercentile(np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values), 99.9))
print(np.nanpercentile(np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values), 0.1))
print(np.nanpercentile(train.life_sq.values/train.full_sq.values, 1))

In [ ]:
life_bad_index=train[(train.life_sq > train.full_sq) & ((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values)) > 2.98 ) & ((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values)) < 5.25 )].index
train.ix[life_bad_index, "life_sq"] = np.NaN
full_bad_index=train[(train.life_sq > train.full_sq) & ((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values)) > 3.07 ) & ((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values)) < 6.45 )].index
train.ix[full_bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > train.full_sq]
bad_index

In [ ]:
life_bad_index=train[(train.life_sq / train.full_sq<0.39) & ((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))>3)&((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))<5.25)].index
train.ix[life_bad_index, "life_sq"] = np.NaN
full_bad_index=train[(train.life_sq / train.full_sq<0.39)&((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values))>3.12)&((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values))<6.46)].index
train.ix[full_bad_index, "full_sq"] = np.NaN
bad_index = train[(train.life_sq / train.full_sq)<0.39]
bad_index

In [ ]:
train=train[((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values)) >3)]

In [ ]:
train=train[((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values)) <6.56)]

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=np.log1p(train.life_sq.values), y=np.log1p(train.price_doc.values), size=10,kind="hex")
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Log of living area in square metre', fontsize=12)
plt.show()

In [ ]:
bad=train[train.full_sq==0]
bad

In [ ]:
bad_index = train[train.life_sq > 300]
bad_index

In [ ]:
bad_index = test[test.life_sq > 200]
bad_index
#test.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN